# Init

In [1]:
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt

In [2]:
%matplotlib qt5

In [3]:
import sys, time

In [4]:
sys.path.insert(0, "/home/pleroy/DEV/processing/PoSAR-MC")

In [5]:
from backprojection.loadbackprojection import *
libraryFilename = "/home/pleroy/DEV/processing/PoSAR-MC/backprojection/ccpp/libbackprojection/liblibbackprojection.so"
lib = LibBackProjection( libraryFilename )

In [6]:
sys.path.insert(0, "/home/pleroy/DEV/DIADEM")
import toolsdiadem.tools as dia

In [7]:
cmap="jet"

# Prepare data

In [8]:
c0 = 3e8
Fmax = 6.44e9
Fmin = 4.6e9
dir_1_couche_sable = "/home/pleroy/DATA/DIADEM/lff/Mesures_sable_Harkati/1_couche_sable"
dir_2_couches = "/home/pleroy/DATA/DIADEM/lff/Mesures_sable_Harkati/2_couches"
dir_scene_vide = "/home/pleroy/DATA/DIADEM/lff/Mesures_sable_Harkati/scene_vide"

In [9]:
data_scene_vide = sio.loadmat(dir_scene_vide + "/RD.mat")
RD_scene_vide = data_scene_vide['RD']

data_1_couche_sable = sio.loadmat(dir_1_couche_sable + "/RD.mat")
RD_1_couche_sable = data_1_couche_sable['RD']

data_2_couches = sio.loadmat(dir_2_couches + "/RD.mat")
RD_2_couches = data_2_couches['RD']

In [10]:
Na0, Nf = RD_1_couche_sable.shape
print(f"Na0 = {Na0}, Nf = {Nf}")

Na0 = 36, Nf = 401


In [11]:
f = np.linspace(Fmin, Fmax, Nf)

In [12]:
kc = 4 * np.pi / c0 * f[int(Nf/2)]
Bf = Fmax - Fmin 
d_rg = c0 / 2 / Bf  #range resolution
print(f"kc = {kc:.3f}, d_rg = {d_rg:.3f}")

kc = 231.221, d_rg = 0.082


In [13]:
im, ax = plt.subplots(1,1)
idx = 1
ax.plot( 20 * np.log10( np.abs( np.fft.ifft( RD_1_couche_sable[idx,:] ) ) ), label=f"1_couche_sable, idx={idx}" )
ax.plot( 20 * np.log10( np.abs( np.fft.ifft( RD_scene_vide[idx,:] ) ) ), label=f"scene_vide, idx={idx}" )
ax.plot( 20 * np.log10( np.abs( np.fft.ifft( RD0[idx,:] ) ) ), label=f"RD0, idx={idx}" )
ax.grid()
ax.legend()

NameError: name 'RD0' is not defined

# Select mode and data

In [14]:
#mode = "biz"
mode = "mono"
#RD0 = np.flipud( RD_1_couche_sable - RD_scene_vide )
RD0 = np.flipud( RD_2_couches - RD_scene_vide )
#RD0 = np.flipud( RD_scene_vide )

H = 3.10 - 0.425 + 0.25    #elevation of antenna #1
d0 = 2.5e-2
NTx = 6

y_Tx = np.ones( (6, 1) ) * 0
z_Tx = (H + 0.425 + np.array( [ 0, 4, 8, 12, 16, 20 ] ) * d0).reshape(-1,1)

if mode == "mono":
    NRx = 5
    # Rx
    x_Rx = np.ones( (NRx, 1) ) * 0
    y_Rx = np.ones( (NRx, 1) ) * 0
    z_Rx = (H + np.array( [ 6, 24, 30, 48, 54 ] ) * d0).reshape(-1,1)
    # Tx
    x_Tx = np.ones( (NTx, 1) ) * 0.25
    # RD
    ind = np.setdiff1d( np.arange(Na0), np.arange(NTx) * NTx )
    RD = RD0[ind,:]
    Na = ind.shape[0]
else:
    NRx = 1
    # Rx
    x_Rx = np.ones( (NRx, 1) ) * 0
    y_Rx = np.ones( (NRx, 1) ) * 6
    z_Rx = np.ones( (NRx, 1) ) * 3.1
    # Tx
    x_Tx = np.ones( (NTx, 1) ) * 0
    # RD
    ind = np.arange(NTx) * NTx
    RD = RD0[ind,:]
    Na = ind.shape[0]
    
print(f"mode = {mode}, Na = {Na}, ind = {ind}")

mode = mono, Na = 30, ind = [ 1  2  3  4  5  7  8  9 10 11 13 14 15 16 17 19 20 21 22 23 25 26 27 28
 29 31 32 33 34 35]


In [15]:
xyzRx = np.zeros( (Na, 5) )
xyzTx = np.zeros( (Na, 5) )

In [16]:
pos = 0

for Tx in range( NTx ):

    for Rx in range( NRx ):
    
        xyzTx[pos, 2] = x_Tx[ Tx ]
        xyzTx[pos, 3] = y_Tx[ Tx ]
        xyzTx[pos, 4] = z_Tx[ Tx ]
        
        xyzRx[pos, 2] = x_Rx[ Rx ]
        xyzRx[pos, 3] = y_Rx[ Rx ]
        xyzRx[pos, 4] = z_Rx[ Rx ]
        
        pos = pos + 1

typedef struct{
    double rampNumber;
    double timeStamp;
    double x;
    double y;
    double z;
} MyPosition;

# Scene parameters

In [17]:
hScene = -1

dx = 0.01
dy = 0.01
dz = 0.01
xMin = 0.125
xMax = 0.125
yMin = 0
yMax = 7
zMin = 0
zMax = 5

x = np.arange(xMin, xMax+1e-9, dx)
y = np.arange(yMin, yMax+1e-9, dy)
z = np.arange(zMin, zMax+1e-9, dz)
nbX = x.size
nbY = y.size
nbZ = z.size

# lefg right bottom top
extent_yz = [yMin, yMax, zMin, zMax]
extent_yx = [yMin, yMax, xMin, xMax]

In [18]:
overSamplingRatio = 10
d_shift = 0
rangeResolution = d_rg
Nover   = overSamplingRatio * Nf
r_base  = np.arange( Nf ) * rangeResolution - d_shift
r_over  = np.arange( Nover ) * rangeResolution / overSamplingRatio - d_shift
dr_over = r_over[1] - r_over[0]

print( f"Nf = {Nf}, Naz = 1, delay = {d_shift}" )
print( f"range from {r_over[0]:.2f}m to {r_over[-1]:.2f}m, "
      + f"resolution = {rangeResolution:.3e}m, oversampled = {rangeResolution / overSamplingRatio:.3e}m" )

Nf = 401, Naz = 1, delay = 0
range from 0.00m to 32.68m, resolution = 8.152e-02m, oversampled = 8.152e-03m


In [19]:
myParameters           = MyParametersPoSAR_GB()
myParameters.Nx        = nbX
myParameters.Ny        = nbY
myParameters.Nz        = nbZ
myParameters.Nover     = r_over.size
myParameters.dx        = dr_over
myParameters.Naz       = 1
myParameters.Nf        = Nf
myParameters.phi_a_deg = 60 # 20° => 7.4cm
myParameters.kc        = kc

# Focusing

In [20]:
t = time.time()

focusedImg = np.zeros( (nbX, nbY, nbZ), dtype=complex )
print( "focusedImg.shape = {}".format( focusedImg.shape ) )
im = np.zeros( (Na, nbY, nbZ), dtype=complex )

for loop in range(Na):
    sr = np.fft.ifftshift( RD[loop,:] )
    focusedImg.fill(0);
    lib.so.backProjectionOmpGroundRange_PoSAR_GB_lha( x, y, z,
                                                     r_over,
                                                     sr.reshape(sr.size),
                                                     xyzRx[loop], xyzTx[loop],
                                                     focusedImg.reshape(focusedImg.size),
                                                     myParameters)
    im[loop,:,:] = focusedImg.squeeze()

elapsed = time.time() - t
print("execution time = " + str(elapsed))

focusedImg.shape = (1, 701, 501)
execution time = 1.0622837543487549


# Plot image

In [21]:
tom = np.sum(im, axis=0)
tom.shape

(701, 501)

In [22]:
toPlot = 10 * np.log10( np.abs(tom)**2 )
#toPlot = np.abs(tom)**2
vmax = np.amax(toPlot)

fig, ax = plt.subplots(1,1)
im = ax.pcolormesh(y, z, toPlot.T, cmap=cmap, vmin=vmax-35, vmax=vmax)
dia.addColorBar( im, ax, aspect="equal" )
ax.set_title("mode " + mode)

Text(0.5, 1.0, 'mode mono')